In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../../")

import os
from copy import copy

import datetime

import numpy as np
import pandas as pd

In [2]:
data_folder = os.path.join("../../","datasets","initial_data_split")

train = pd.read_csv(os.path.join(data_folder,"train.csv"))
print train.head(5)

              datetime  season  holiday  workingday  weather  temp   atemp  \
0  2011-01-01 00:00:00       1        0           0        1  9.84  14.395   
1  2011-01-01 01:00:00       1        0           0        1  9.02  13.635   
2  2011-01-01 02:00:00       1        0           0        1  9.02  13.635   
3  2011-01-01 03:00:00       1        0           0        1  9.84  14.395   
4  2011-01-01 04:00:00       1        0           0        1  9.84  14.395   

   humidity  windspeed  casual  registered  count  
0        81        0.0       3          13     16  
1        80        0.0       8          32     40  
2        80        0.0       5          27     32  
3        75        0.0       3          10     13  
4        75        0.0       0           1      1  


In [22]:
example_datetime = train[["datetime"]].values[10][0]
print example_datetime

2011-01-01 10:00:00


In [23]:
example_datetime = datetime.datetime.strptime(example_datetime, "%Y-%m-%d %H:%M:%S")

In [24]:
print example_datetime.weekday()
print example_datetime.year
print example_datetime.month
print example_datetime.hour

5
2011
1
10


In [25]:
new_train = train
new_train['datetime'] =  pd.to_datetime(new_train['datetime'], format="%Y-%m-%d %H:%M:%S")
new_train['time_hour'] = new_train['datetime'].apply(lambda x: x.hour)
new_train['date_day'] = new_train['datetime'].apply(lambda x: x.weekday())
new_train['date_weekday'] = new_train['datetime'].apply(lambda x: x.weekday())
new_train['date_month'] = new_train['datetime'].apply(lambda x: x.month)
new_train['date_year'] = new_train['datetime'].apply(lambda x: x.year)

In [54]:
from statistics import mode
def safe_mode(x):
    try:
        return mode(x)
    except Exception:
        return np.nan

new_train["lagging_freq"] = pd.Series.rolling(new_train["weather"],
                                              window = 10,min_periods=1).apply(lambda x: safe_mode(x))
# pd.rolling_apply
print new_train[["lagging_freq","weather","time_hour","datetime"]]

      lagging_freq  weather  time_hour            datetime
0              1.0        1          0 2011-01-01 00:00:00
1              1.0        1          1 2011-01-01 01:00:00
2              1.0        1          2 2011-01-01 02:00:00
3              1.0        1          3 2011-01-01 03:00:00
4              1.0        1          4 2011-01-01 04:00:00
5              1.0        2          5 2011-01-01 05:00:00
6              1.0        1          6 2011-01-01 06:00:00
7              1.0        1          7 2011-01-01 07:00:00
8              1.0        1          8 2011-01-01 08:00:00
9              1.0        1          9 2011-01-01 09:00:00
10             1.0        1         10 2011-01-01 10:00:00
11             1.0        1         11 2011-01-01 11:00:00
12             1.0        1         12 2011-01-01 12:00:00
13             1.0        2         13 2011-01-01 13:00:00
14             1.0        2         14 2011-01-01 14:00:00
15             1.0        2         15 2011-01-01 15:00:

In [15]:
print new_train.head(5)

             datetime  season  holiday  workingday  weather  temp   atemp  \
0 2011-01-01 00:00:00       1        0           0        1  9.84  14.395   
1 2011-01-01 01:00:00       1        0           0        1  9.02  13.635   
2 2011-01-01 02:00:00       1        0           0        1  9.02  13.635   
3 2011-01-01 03:00:00       1        0           0        1  9.84  14.395   
4 2011-01-01 04:00:00       1        0           0        1  9.84  14.395   

   humidity  windspeed  casual  registered  count  time_hour  date_day  \
0        81        0.0       3          13     16          0         5   
1        80        0.0       8          32     40          1         5   
2        80        0.0       5          27     32          2         5   
3        75        0.0       3          10     13          3         5   
4        75        0.0       0           1      1          4         5   

   date_weekday  date_month  date_year  
0             5           1       2011  
1         

In [16]:
average_train = train
average_train["atemp_average_20"] = pd.Series.rolling(average_train["temp"],window = 20,min_periods=1).mean()
average_train = average_train.fillna(method="bfill")
print average_train.head(30)

              datetime  season  holiday  workingday  weather   temp   atemp  \
0  2011-01-01 00:00:00       1        0           0        1   9.84  14.395   
1  2011-01-01 01:00:00       1        0           0        1   9.02  13.635   
2  2011-01-01 02:00:00       1        0           0        1   9.02  13.635   
3  2011-01-01 03:00:00       1        0           0        1   9.84  14.395   
4  2011-01-01 04:00:00       1        0           0        1   9.84  14.395   
5  2011-01-01 05:00:00       1        0           0        2   9.84  12.880   
6  2011-01-01 06:00:00       1        0           0        1   9.02  13.635   
7  2011-01-01 07:00:00       1        0           0        1   8.20  12.880   
8  2011-01-01 08:00:00       1        0           0        1   9.84  14.395   
9  2011-01-01 09:00:00       1        0           0        1  13.12  17.425   
10 2011-01-01 10:00:00       1        0           0        1  15.58  19.695   
11 2011-01-01 11:00:00       1        0           0 

In [17]:
from utils.preprocessing_utils import generate_average_timelag_data
test_average_train=train

test_average_train = generate_average_timelag_data(test_average_train,["weather","atemp"],lag=10)

print test_average_train.head(30)

ImportError: cannot import name generate_average_timelag_data

In [18]:
new_train = train
def is_day_or_night(hour):
    if hour>6 and hour<21:
        return 1
    else:
        return 0

new_train["day_night"] = new_train['time_hour'].apply(lambda x: is_day_or_night(x))

print new_train.head(10)

             datetime  season  holiday  workingday  weather   temp   atemp  \
0 2011-01-01 00:00:00       1        0           0        1   9.84  14.395   
1 2011-01-01 01:00:00       1        0           0        1   9.02  13.635   
2 2011-01-01 02:00:00       1        0           0        1   9.02  13.635   
3 2011-01-01 03:00:00       1        0           0        1   9.84  14.395   
4 2011-01-01 04:00:00       1        0           0        1   9.84  14.395   
5 2011-01-01 05:00:00       1        0           0        2   9.84  12.880   
6 2011-01-01 06:00:00       1        0           0        1   9.02  13.635   
7 2011-01-01 07:00:00       1        0           0        1   8.20  12.880   
8 2011-01-01 08:00:00       1        0           0        1   9.84  14.395   
9 2011-01-01 09:00:00       1        0           0        1  13.12  17.425   

   humidity  windspeed  casual  registered  count  time_hour  date_day  \
0        81     0.0000       3          13     16          0       

In [19]:
def lagged_day_night_min(data,colnames,lag,mode = "day"):
    trans_name = "min"
    edge_drop_name = "edge_drop_lag_%s"%lag
    data["day_night"] = data['time_hour'].apply(lambda x: is_day_or_night(x))
    data[edge_drop_name] = data['datetime'].apply(lambda x: is_day_or_night(x))
    if mode=="day":
        mode_value=1
        
    elif mode=="night":
        mode_value=0
    else:
        raise Exception("Wrong mode")
        
    full_result = data.set_index("datetime",drop=False)
    for colname in colnames:
        new_colname = colname+"_%s_%s_%s"%(mode,trans_name,lag)
        g= data.set_index('day_night', append=True,drop=False).groupby(level=1)
        colname_result = pd.DataFrame()
        for k, gp in g:
            if k==mode_value:
                gp[new_colname] = pd.Series.rolling(gp[colname],
                                                       window=lag,
                                                       min_periods=1).min()
            colname_result = pd.concat([colname_result,gp])
        colname_result = colname_result.set_index("datetime",drop=False)
        full_result[new_colname] = colname_result[new_colname]

    full_result = full_result.sort_values(["datetime"]).fillna(method="ffill")
    for colname in colnames:
        new_colname = colname+"_%s_%s_%s"%(mode,trans_name,lag)
        full_result[new_colname] = full_result[new_colname].fillna(full_result[colname])
    return full_result

# smth =  lagged_day_night_mean(new_train,["temp","weather","atemp"],lag=3,mode = "day")
smth =  lagged_day_night_min(train,["temp","weather","atemp"],lag=7,mode = "day")

print smth.head(30)

TypeError: Cannot compare type 'Timestamp' with type 'int'

In [140]:
from utils.preprocessing_utils import lagged_day_night_span,lagged_day_night_quantile_variation

smth = lagged_day_night_span(train,
                                    colnames = ["temp","atemp"],
                                    lag=5,mode="day")

smth = lagged_day_night_quantile_variation(smth,
                                    colnames = ["temp","atemp"],
                                    lag=5,mode="day")

print smth[['atemp_day_span_5','atemp','day_night']].head(100)

                     atemp_day_span_5   atemp  day_night
datetime                                                
2011-01-01 00:00:00             0.000  14.395          0
2011-01-01 01:00:00             0.000  13.635          0
2011-01-01 02:00:00             0.000  13.635          0
2011-01-01 03:00:00             0.000  14.395          0
2011-01-01 04:00:00             0.000  14.395          0
2011-01-01 05:00:00             0.000  12.880          0
2011-01-01 06:00:00             0.000  13.635          0
2011-01-01 07:00:00             0.000  12.880          1
2011-01-01 08:00:00             0.000  14.395          1
2011-01-01 09:00:00             0.000  17.425          1
2011-01-01 10:00:00             0.000  19.695          1
2011-01-01 11:00:00             6.815  16.665          1
2011-01-01 12:00:00             6.815  21.210          1
2011-01-01 13:00:00             6.060  22.725          1
2011-01-01 14:00:00             6.060  22.725          1
2011-01-01 15:00:00            

In [121]:
print smth.columns

Index([u'datetime', u'season', u'holiday', u'workingday', u'weather', u'temp',
       u'atemp', u'humidity', u'windspeed', u'casual', u'registered', u'count',
       u'time_hour', u'date_weekday', u'date_month', u'date_year',
       u'day_night', u'temp_day_lag_7', u'weather_day_lag_7',
       u'atemp_day_lag_7'],
      dtype='object')


In [16]:
from utils.preprocessing_utils import preprocess_rolling,preprocess_datetime
from utils.generic_utils import pickle_in

rolling_vars_filepath = os.path.join("../../","models","varnames_rolling.pkl")
rolling_important_vars = pickle_in(rolling_vars_filepath,compresion_mode=0)

subset = train[:100]
subset = preprocess_datetime(subset)

subs_X,subsY = subset

rolling_lags = [5,12,24,50,100]
variables_to_roll = ["temp","humidity","windspeed"]

subset_rolling = preprocess_rolling(subs_X,
                                    important_variables = rolling_important_vars,
                                    rolling_lags = rolling_lags,
                                    variables_to_roll = variables_to_roll)

['day_night' 'time_hour' 'date_year' 'workingday'
 'temp_night_quantile_25_100' 'date_weekday' 'temp_day_quantile_75_5'
 'date_month' 'temp_day_quantile_25_100' 'humidity_day_max_5'
 'temp_day_median_5' 'humidity' 'atemp' 'weather' 'temp'
 'humidity_day_median_5' 'temp_day_min_5' 'temp_night_median_100'
 'humidity_day_quantile_75_12' 'humidity_day_span_5'
 'humidity_day_quantile_75_5' 'humidity_day_quantile_25_5'
 'temp_day_quantile_25_5' 'temp_day_max_5' 'humidity_night_span_5'
 'temp_day_median_24' 'temp_day_median_12' 'windspeed_day_span_5'
 'temp_day_span_5' 'humidity_day_span_12' 'windspeed_day_median_12'
 'temp_day_quantile_75_100' 'temp_day_quantile_75_12' 'temp_day_max_100'
 'humidity_day_median_12' 'windspeed' 'humidity_day_span_100'
 'windspeed_day_quantile_75_5' 'temp_day_quantile_25_12'
 'temp_night_quantile_25_50' 'humidity_day_quantile_75_24'
 'windspeed_day_span_12' 'temp_day_span_12' 'humidity_day_min_5'
 'humidity_day_span_24' 'windspeed_day_max_5' 'temp_night_quantile

../../utils/preprocessing_utils.py:303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  min_periods=1).median()
../../utils/preprocessing_utils.py:334: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  min_periods=1).max()
../../utils/preprocessing_utils.py:365: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  min_periods=1).m


temp
humidity
windspeed
temp
humidity
windspeed
temp
humidity
windspeed
temp
humidity
windspeed
temp
humidity
windspeed
temp
humidity
windspeed
temp
humidity
windspeed
temp
humidity
windspeed


In [41]:
from utils.preprocessing_utils import preprocess_rolling,preprocess_datetime
subset = train
subset = preprocess_datetime(subset)

subs_X,subsY = subset

In [47]:
def add_edge_gap(data,lag):
    result = data
    result["time_diff"] = result["datetime"].diff()
    result["time_diff"] = pd.to_numeric(result["time_diff"])/3600000000000
    result["max_in_lag"] = pd.Series.rolling(result["time_diff"],window=lag).max()
    result["max_in_lag"] = result["max_in_lag"].fillna(30)
    result["edge_lag_%s"%lag] = result["max_in_lag"].apply(lambda x:x>24)
    result["edge_lag_%s"%lag] =  result["edge_lag_%s"%lag]*1.0
    result.drop(["time_diff","max_in_lag"],axis=1)
    
    return result

output = add_edge_gap(subs_X,lag=10)
print output["edge_lag_10"].describe()
print output.head()

Index([u'datetime', u'season', u'holiday', u'workingday', u'weather', u'temp',
       u'atemp', u'humidity', u'windspeed', u'time_hour', u'date_weekday',
       u'date_day', u'date_month', u'date_year', u'time_diff', u'max_in_lag',
       u'edge_lag_10'],
      dtype='object')
count    7452.000000
mean        0.032072
std         0.176203
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: edge_lag_10, dtype: float64
             datetime  season  holiday  workingday  weather  temp   atemp  \
0 2011-01-01 00:00:00       1        0           0        1  9.84  14.395   
1 2011-01-01 01:00:00       1        0           0        1  9.02  13.635   
2 2011-01-01 02:00:00       1        0           0        1  9.02  13.635   
3 2011-01-01 03:00:00       1        0           0        1  9.84  14.395   
4 2011-01-01 04:00:00       1        0           0        1  9.84  14.395   

   humidity  windspeed  time_hour  date_weekday  date_day

In [49]:
from utils.preprocessing_utils import preprocess_rolling,preprocess_datetime,preprocess_rolling_edge

output = preprocess_rolling_edge(subs_X,[5,10,6,29])
print output.head()

             datetime  season  holiday  workingday  weather  temp   atemp  \
0 2011-01-01 00:00:00       1        0           0        1  9.84  14.395   
1 2011-01-01 01:00:00       1        0           0        1  9.02  13.635   
2 2011-01-01 02:00:00       1        0           0        1  9.02  13.635   
3 2011-01-01 03:00:00       1        0           0        1  9.84  14.395   
4 2011-01-01 04:00:00       1        0           0        1  9.84  14.395   

   humidity  windspeed  time_hour  date_weekday  date_day  date_month  \
0        81        0.0          0             5         1           1   
1        80        0.0          1             5         1           1   
2        80        0.0          2             5         1           1   
3        75        0.0          3             5         1           1   
4        75        0.0          4             5         1           1   

   date_year     time_diff  max_in_lag  edge_lag_10  edge_lag_5  edge_lag_6  \
0       2011 -2.562

In [113]:
from copy import copy
data_folder = os.path.join("../../","datasets","initial_data_split")

train = pd.read_csv(os.path.join(data_folder,"train.csv"))
train1 = train.copy()

In [119]:
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.preprocessing import LabelEncoder
from numpy.random import randint

class CleanCategorical(BaseEstimator, TransformerMixin):
    def __init__(self,colname):
        self.colname = colname
        self.label_encoder = LabelEncoder()
        
    def fit(self, X, y=None):
        self.label_encoder.fit(X[self.colname])
        return self
    
    def transform(self, X, y=None, copy=None):
        relabeled = self.label_encoder.transform(X[self.colname])
        X[self.colname] = X[self.colname].replace(relabeled)
        return X
    
class CleanCategoricalSimple(BaseEstimator, TransformerMixin):
    def __init__(self,colname):
        self.colname = colname
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None, copy=None):
        X_out = X.copy()
#         X.loc[:,self.colname] = randint(0,100,1)
        X_out[self.colname] = X[self.colname]-1
        return X_out

In [120]:
ccs=CleanCategoricalSimple(colname='season')
ccs.fit(train)

CleanCategoricalSimple(colname='season')

In [122]:
res1 = ccs.transform(train)
print res1['season'].describe()
print train['season'].describe()

count    7452.000000
mean        1.505636
std         1.116653
min         0.000000
25%         1.000000
50%         2.000000
75%         3.000000
max         3.000000
Name: season, dtype: float64
count    7452.000000
mean        1.505636
std         1.116653
min         0.000000
25%         1.000000
50%         2.000000
75%         3.000000
max         3.000000
Name: season, dtype: float64
count    7452.000000
mean        2.505636
std         1.116653
min         1.000000
25%         2.000000
50%         3.000000
75%         4.000000
max         4.000000
Name: season, dtype: float64


In [118]:
train1['season']=train['season']-1
print train1["season"].describe()
print train["season"].describe()

count    7452.000000
mean        1.505636
std         1.116653
min         0.000000
25%         1.000000
50%         2.000000
75%         3.000000
max         3.000000
Name: season, dtype: float64
count    7452.000000
mean        2.505636
std         1.116653
min         1.000000
25%         2.000000
50%         3.000000
75%         4.000000
max         4.000000
Name: season, dtype: float64


In [62]:
print train1.shape,result.shape,train.shape
print result["season"].describe()
print train1["season"].describe()

(7452, 13) (7452, 13) (7452, 13)
count    7452.000000
mean        1.505636
std         1.116653
min         0.000000
25%         1.000000
50%         2.000000
75%         3.000000
max         3.000000
Name: season, dtype: float64
count    7452.000000
mean        1.505636
std         1.116653
min         0.000000
25%         1.000000
50%         2.000000
75%         3.000000
max         3.000000
Name: season, dtype: float64


In [21]:
train_binning = train.copy()
print train_binning[['atemp','humidity','windspeed']].describe()

             atemp     humidity    windspeed
count  7452.000000  7452.000000  7452.000000
mean     23.487207    61.732152    12.703198
std       8.823987    19.785894     8.268141
min       0.760000     0.000000     0.000000
25%      15.910000    46.000000     7.001500
50%      23.485000    61.000000    11.001400
75%      31.060000    78.000000    16.997900
max      45.455000   100.000000    56.996900


In [22]:
temp_bins = [10,20,30]
train_binning["atemp_binned"] = pd.cut(train_binning['atemp'],bins = temp_bins)
humidity_bins = [20,40,60,80]
train_binning["humidity_binned"] = pd.cut(train_binning['humidity'],bins = humidity_bins)
print train_binning

print train_binning

                 datetime  season  holiday  workingday  weather   temp  \
0     2011-01-01 00:00:00       1        0           0        1   9.84   
1     2011-01-01 01:00:00       1        0           0        1   9.02   
2     2011-01-01 02:00:00       1        0           0        1   9.02   
3     2011-01-01 03:00:00       1        0           0        1   9.84   
4     2011-01-01 04:00:00       1        0           0        1   9.84   
5     2011-01-01 05:00:00       1        0           0        2   9.84   
6     2011-01-01 06:00:00       1        0           0        1   9.02   
7     2011-01-01 07:00:00       1        0           0        1   8.20   
8     2011-01-01 08:00:00       1        0           0        1   9.84   
9     2011-01-01 09:00:00       1        0           0        1  13.12   
10    2011-01-01 10:00:00       1        0           0        1  15.58   
11    2011-01-01 11:00:00       1        0           0        1  14.76   
12    2011-01-01 12:00:00       1     

In [32]:
from utils.preprocessing_utils import BinVariable
train_binning = train.copy()

atemp_binner = BinVariable(colname = "atemp",bins  = [0,10,20,30,100],drop_column = False)
humidity_binner = BinVariable(colname = "humidity",bins  = [0,20,40,60,120],drop_column = False)
wind_binner = BinVariable(colname = "windspeed",bins  = [0,30,1000],drop_column = False)
train_binning = atemp_binner.fit_transform(train_binning)
train_binning = humidity_binner.fit_transform(train_binning)
train_binning = wind_binner.fit_transform(train_binning)
print train_binning.head()

              datetime  season  holiday  workingday  weather  temp   atemp  \
0  2011-01-01 00:00:00       1        0           0        1  9.84  14.395   
1  2011-01-01 01:00:00       1        0           0        1  9.02  13.635   
2  2011-01-01 02:00:00       1        0           0        1  9.02  13.635   
3  2011-01-01 03:00:00       1        0           0        1  9.84  14.395   
4  2011-01-01 04:00:00       1        0           0        1  9.84  14.395   

   humidity  windspeed  casual  registered  count atemp_binned  \
0        81        0.0       3          13     16     (10, 20]   
1        80        0.0       8          32     40     (10, 20]   
2        80        0.0       5          27     32     (10, 20]   
3        75        0.0       3          10     13     (10, 20]   
4        75        0.0       0           1      1     (10, 20]   

  humidity_binned windspeed_binned  
0       (60, 120]          [0, 30]  
1       (60, 120]          [0, 30]  
2       (60, 120]      

In [ ]:
print train_binning["humidity_binned"]